# Tutorial 1 - RNN Sequence classifier

In this notebook, we will predict the winner of a basketball game based on the scores observed in the first 3 quarters of the game. Each column represents the beginning of a minute during the game. (There are 12 minutes in each quarter. There are  3 quarters in the data, so we have 36 columns as input variables in chronological order.) The values captured in each column represent the score difference observed at that minute (home score minus away score)<br><br>

The last column `W` represents whether the home team (1) or the away team (0) won the game. This is the target variable. <br><br>
**Our unit of analysis is a single game.**

In [1]:
# Common imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

np.random.seed(1)
tf.random.set_seed(1)

## Get the data

In [3]:
data = pd.read_csv("basketball.csv")

In [4]:
data.shape

(1230, 37)

In [5]:
data.head()

,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,...,M28,M29,M30,M31,M32,M33,M34,M35,M36,W
0,-2,-1,1,1,-1,-3,-1,0,3,6,...,9,11,10,7,7,4,6,2,1,1
1,0,2,7,6,10,8,8,6,0,6,...,7,11,11,14,14,15,13,13,13,0
2,0,-2,2,0,5,4,5,3,5,3,...,9,11,13,13,12,17,15,15,12,1
3,0,2,0,3,4,3,5,4,3,1,...,10,6,7,7,8,8,8,8,8,1
4,0,-2,-2,0,3,-2,-7,-5,-7,-4,...,10,10,15,13,11,11,11,13,10,1


In [6]:
y = data['W']
X = data.drop('W', axis=1)

## Split the data

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Data Transformation

In [8]:
#Target variables need to be an array with integer type
y_train = np.array(y_train)
y_test = np.array(y_test)

y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

In [9]:
#Check the first 10 values of the train_y data set
y_train[0:10]

array([0, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [10]:
#Convert input variables to a 2-D array with float data type
X_train = np.array(X_train)
X_test = np.array(X_test)

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [11]:
X_train

array([[ -3.,  -3.,  -2., ...,  -5.,  -3.,  -7.],
       [  0.,   1.,  -1., ...,  10.,  12.,  15.],
       [ -3.,  -2.,  -2., ...,  14.,  12.,  12.],
       ...,
       [  2.,   6.,   8., ...,  -2.,   1.,   0.],
       [ -2.,   1.,  -1., ..., -19., -21., -14.],
       [  2.,   1.,   3., ...,   0.,   2.,   4.]], dtype=float32)

In [12]:
#Keras expects a different input format:
#Data needs to have 3 dimensions

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [13]:
X_train.shape, y_train.shape

((861, 36, 1), (861,))

In [14]:
X_train

array([[[ -3.],
        [ -3.],
        [ -2.],
        ...,
        [ -5.],
        [ -3.],
        [ -7.]],

       [[  0.],
        [  1.],
        [ -1.],
        ...,
        [ 10.],
        [ 12.],
        [ 15.]],

       [[ -3.],
        [ -2.],
        [ -2.],
        ...,
        [ 14.],
        [ 12.],
        [ 12.]],

       ...,

       [[  2.],
        [  6.],
        [  8.],
        ...,
        [ -2.],
        [  1.],
        [  0.]],

       [[ -2.],
        [  1.],
        [ -1.],
        ...,
        [-19.],
        [-21.],
        [-14.]],

       [[  2.],
        [  1.],
        [  3.],
        ...,
        [  0.],
        [  2.],
        [  4.]]], dtype=float32)

# A normal (cross-sectional) NN

This model assumes that the data is NOT a time-series data set. It treats the data as cross-sectional and the columns being independent of each other.

In [15]:
model = keras.models.Sequential([
    
    keras.layers.Flatten(input_shape=[36, 1]),
    keras.layers.Dense(36, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
    
])

In [16]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=50,
                    validation_data=(X_test, y_test))

Epoch 1/50
27/27 [==============================] - 5s 22ms/step - loss: 0.8406 - accuracy: 0.7271 - val_loss: 0.5258 - val_accuracy: 0.7859
Epoch 2/50
27/27 [==============================] - 0s 5ms/step - loss: 0.5658 - accuracy: 0.7724 - val_loss: 0.5487 - val_accuracy: 0.7886
Epoch 3/50
27/27 [==============================] - 0s 6ms/step - loss: 0.4437 - accuracy: 0.7863 - val_loss: 0.4938 - val_accuracy: 0.8076
Epoch 4/50
27/27 [==============================] - 0s 6ms/step - loss: 0.4089 - accuracy: 0.8095 - val_loss: 0.4881 - val_accuracy: 0.8157
Epoch 5/50
27/27 [==============================] - 0s 6ms/step - loss: 0.3892 - accuracy: 0.8188 - val_loss: 0.4739 - val_accuracy: 0.7886
Epoch 6/50
27/27 [==============================] - 0s 6ms/step - loss: 0.4008 - accuracy: 0.8060 - val_loss: 0.5862 - val_accuracy: 0.7046
Epoch 7/50
27/27 [==============================] - 0s 6ms/step - loss: 0.3665 - accuracy: 0.8304 - val_loss: 0.5044 - val_accuracy: 0.7995
Epoch 8/50
27/27 [=

In [17]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.7698073387145996, 0.7886179089546204]

In [18]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.77
accuracy: 78.86%


# Simple RNN with one layer

In [19]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.SimpleRNN(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [20]:
from tensorflow.keras.callbacks import EarlyStopping

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

history = model.fit(
    X_train, 
    y_train, 
    epochs=50,
    validation_data=(X_test, y_test), 
    callbacks=[early_stop]
)

Epoch 1/50
27/27 [==============================] - 4s 37ms/step - loss: 0.4336 - accuracy: 0.8072 - val_loss: 0.3965 - val_accuracy: 0.8238
Epoch 2/50
27/27 [==============================] - 0s 15ms/step - loss: 0.4139 - accuracy: 0.8037 - val_loss: 0.4102 - val_accuracy: 0.7995
Epoch 3/50
27/27 [==============================] - 0s 15ms/step - loss: 0.4073 - accuracy: 0.8002 - val_loss: 0.4058 - val_accuracy: 0.8076
Epoch 4/50
27/27 [==============================] - 0s 15ms/step - loss: 0.4054 - accuracy: 0.8026 - val_loss: 0.3941 - val_accuracy: 0.8266
Epoch 5/50
27/27 [==============================] - 0s 15ms/step - loss: 0.3975 - accuracy: 0.8211 - val_loss: 0.3947 - val_accuracy: 0.8076
Epoch 6/50
27/27 [==============================] - 0s 15ms/step - loss: 0.3878 - accuracy: 0.8153 - val_loss: 0.4024 - val_accuracy: 0.8049
Epoch 7/50
27/27 [==============================] - 0s 16ms/step - loss: 0.3888 - accuracy: 0.8153 - val_loss: 0.3966 - val_accuracy: 0.8184
Epoch 8/50
27

In [21]:
# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.40837013721466064, 0.8075881004333496]

In [22]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.41
accuracy: 80.76


In [23]:
# Predictions are probabilities.

predictions = model.predict(X_test)

12/12 [==============================] - 0s 5ms/step


In [24]:
# Rounding the probabilities determines 1 or 0

np.round(predictions)[:10] # show first 10 predictions

array([[0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.]], dtype=float32)

In [25]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, np.round(predictions))

array([[129,  38],
       [ 33, 169]], dtype=int64)

# Deep RNN

**Be careful: when stacking RNN layers, you have to set "return_sequences" to True. This enables the layer to send a "sequence" of values to the next layer -- jut like how it uses a sequence of values for training.**

**Since the last layer is DENSE, it can't take sequence data. Therefore, you CANNOT return sequences from the previous layer. So, remove** `return_sequences` **from previous layer.**

In [26]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.SimpleRNN(32, return_sequences=True, input_shape=[n_steps, n_inputs] ),
    keras.layers.SimpleRNN(32, return_sequences=True),
    keras.layers.SimpleRNN(32), 
    keras.layers.Dense(1, activation='sigmoid')
])

In [28]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data = (X_test, y_test), 
    callbacks=[early_stop])  # we defined this early_stop callback function earlier

Epoch 1/20
27/27 [==============================] - 9s 68ms/step - loss: 0.4145 - accuracy: 0.8084 - val_loss: 0.4017 - val_accuracy: 0.8238
Epoch 2/20
27/27 [==============================] - 1s 35ms/step - loss: 0.4032 - accuracy: 0.8049 - val_loss: 0.4144 - val_accuracy: 0.7995
Epoch 3/20
27/27 [==============================] - 1s 34ms/step - loss: 0.4110 - accuracy: 0.7967 - val_loss: 0.4057 - val_accuracy: 0.8049
Epoch 4/20
27/27 [==============================] - 1s 31ms/step - loss: 0.3873 - accuracy: 0.8118 - val_loss: 0.4001 - val_accuracy: 0.8293
Epoch 5/20
27/27 [==============================] - 1s 35ms/step - loss: 0.3804 - accuracy: 0.8153 - val_loss: 0.4054 - val_accuracy: 0.7995
Epoch 6/20
27/27 [==============================] - 1s 35ms/step - loss: 0.3775 - accuracy: 0.8153 - val_loss: 0.4216 - val_accuracy: 0.7967
Epoch 7/20
27/27 [==============================] - 1s 36ms/step - loss: 0.3695 - accuracy: 0.8200 - val_loss: 0.3998 - val_accuracy: 0.8266
Epoch 8/20
27

In [29]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)

scores
# In results, first is loss, second is accuracy

[0.49157360196113586, 0.7344173192977905]

In [30]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.49
accuracy: 73.44


# LSTM with one layer

In [31]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [32]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test),
    callbacks=[early_stop]) # we defined this early_stop callback function earlier

Epoch 1/20
27/27 [==============================] - 7s 71ms/step - loss: 0.4579 - accuracy: 0.7851 - val_loss: 0.3890 - val_accuracy: 0.8320
Epoch 2/20
27/27 [==============================] - 1s 27ms/step - loss: 0.4167 - accuracy: 0.8014 - val_loss: 0.4066 - val_accuracy: 0.8103
Epoch 3/20
27/27 [==============================] - 1s 27ms/step - loss: 0.4136 - accuracy: 0.8026 - val_loss: 0.3934 - val_accuracy: 0.8238
Epoch 4/20
27/27 [==============================] - 1s 27ms/step - loss: 0.4096 - accuracy: 0.8049 - val_loss: 0.3910 - val_accuracy: 0.8320
Epoch 5/20
27/27 [==============================] - 1s 27ms/step - loss: 0.4039 - accuracy: 0.8130 - val_loss: 0.3988 - val_accuracy: 0.8401
Epoch 6/20
27/27 [==============================] - 1s 27ms/step - loss: 0.3984 - accuracy: 0.8049 - val_loss: 0.4004 - val_accuracy: 0.8211
Epoch 6: early stopping


In [33]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.4003751873970032, 0.8211382031440735]

In [34]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.40
accuracy: 82.11


# LSTM with more layers

In [35]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(32, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(1, activation='sigmoid')
])

In [36]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test), 
    callbacks=[early_stop] # we defined this early_stop callback function earlier
)

Epoch 1/20
27/27 [==============================] - 17s 162ms/step - loss: 0.5060 - accuracy: 0.7619 - val_loss: 0.4156 - val_accuracy: 0.8238
Epoch 2/20
27/27 [==============================] - 2s 73ms/step - loss: 0.4383 - accuracy: 0.7921 - val_loss: 0.4210 - val_accuracy: 0.8022
Epoch 3/20
27/27 [==============================] - 2s 74ms/step - loss: 0.4233 - accuracy: 0.7944 - val_loss: 0.4132 - val_accuracy: 0.8022
Epoch 4/20
27/27 [==============================] - 2s 71ms/step - loss: 0.4137 - accuracy: 0.7991 - val_loss: 0.3955 - val_accuracy: 0.8238
Epoch 5/20
27/27 [==============================] - 2s 71ms/step - loss: 0.4152 - accuracy: 0.8026 - val_loss: 0.4226 - val_accuracy: 0.7967
Epoch 6/20
27/27 [==============================] - 2s 73ms/step - loss: 0.4095 - accuracy: 0.7991 - val_loss: 0.4097 - val_accuracy: 0.8211
Epoch 7/20
27/27 [==============================] - 2s 73ms/step - loss: 0.3989 - accuracy: 0.7816 - val_loss: 0.4296 - val_accuracy: 0.7967
Epoch 8/20


In [37]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.4477121829986572, 0.794037938117981]

In [38]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.45
accuracy: 79.40%


# GRU with one layer

In [39]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [40]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test),
    callbacks=[early_stop] # we defined this early_stop callback function earlier
)

Epoch 1/20
27/27 [==============================] - 7s 64ms/step - loss: 0.4654 - accuracy: 0.7782 - val_loss: 0.3886 - val_accuracy: 0.8266
Epoch 2/20
27/27 [==============================] - 1s 29ms/step - loss: 0.4159 - accuracy: 0.7979 - val_loss: 0.4055 - val_accuracy: 0.8130
Epoch 3/20
27/27 [==============================] - 1s 29ms/step - loss: 0.4183 - accuracy: 0.8037 - val_loss: 0.3906 - val_accuracy: 0.8320
Epoch 4/20
27/27 [==============================] - 1s 28ms/step - loss: 0.4104 - accuracy: 0.7944 - val_loss: 0.3884 - val_accuracy: 0.8238
Epoch 5/20
27/27 [==============================] - 1s 28ms/step - loss: 0.4077 - accuracy: 0.8037 - val_loss: 0.3951 - val_accuracy: 0.8320
Epoch 6/20
27/27 [==============================] - 1s 26ms/step - loss: 0.4038 - accuracy: 0.7967 - val_loss: 0.4000 - val_accuracy: 0.8266
Epoch 7/20
27/27 [==============================] - 1s 28ms/step - loss: 0.4008 - accuracy: 0.7991 - val_loss: 0.3986 - val_accuracy: 0.8266
Epoch 8/20
27

In [41]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.40662381052970886, 0.8102980852127075]

In [42]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.41
accuracy: 81.03


# GRU with more layers

In [43]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(32, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(32, return_sequences=True),
    keras.layers.GRU(32),
    keras.layers.Dense(1, activation='sigmoid')
])

In [44]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test), 
    callbacks=[early_stop] # we defined this early_stop callback function earlier
)

Epoch 1/20
27/27 [==============================] - 17s 162ms/step - loss: 0.4543 - accuracy: 0.7933 - val_loss: 0.3890 - val_accuracy: 0.8211
Epoch 2/20
27/27 [==============================] - 2s 80ms/step - loss: 0.4204 - accuracy: 0.7898 - val_loss: 0.4030 - val_accuracy: 0.8157
Epoch 3/20
27/27 [==============================] - 2s 80ms/step - loss: 0.4199 - accuracy: 0.7991 - val_loss: 0.4047 - val_accuracy: 0.8184
Epoch 4/20
27/27 [==============================] - 2s 76ms/step - loss: 0.4120 - accuracy: 0.7921 - val_loss: 0.3907 - val_accuracy: 0.8157
Epoch 5/20
27/27 [==============================] - 2s 80ms/step - loss: 0.4164 - accuracy: 0.7956 - val_loss: 0.4076 - val_accuracy: 0.8130
Epoch 6/20
27/27 [==============================] - 2s 80ms/step - loss: 0.4119 - accuracy: 0.7944 - val_loss: 0.4112 - val_accuracy: 0.8266
Epoch 6: early stopping


In [45]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.4112195670604706, 0.8265582919120789]

In [46]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.41
accuracy: 82.66%


# Conv1D

In [47]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.Conv1D(filters=10, kernel_size=3, strides=2, padding="valid", input_shape=[n_steps, n_inputs]),
    keras.layers.Conv1D(filters=20, kernel_size=3, strides=1, padding="valid", dilation_rate=2),
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 17, 10)            40        
                                                                 
 conv1d_1 (Conv1D)           (None, 13, 20)            620       
                                                                 
 lstm_4 (LSTM)               (None, 13, 32)            6784      
                                                                 
 lstm_5 (LSTM)               (None, 32)                8320      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 15,797
Trainable params: 15,797
Non-trainable params: 0
_________________________________________________________________


In [48]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test), 
    callbacks=[early_stop] # we defined this early_stop callback function earlier
)

Epoch 1/20
27/27 [==============================] - 13s 90ms/step - loss: 0.5281 - accuracy: 0.7282 - val_loss: 0.4299 - val_accuracy: 0.8238
Epoch 2/20
27/27 [==============================] - 1s 27ms/step - loss: 0.4503 - accuracy: 0.7782 - val_loss: 0.4259 - val_accuracy: 0.8157
Epoch 3/20
27/27 [==============================] - 1s 27ms/step - loss: 0.4321 - accuracy: 0.7875 - val_loss: 0.4115 - val_accuracy: 0.7995
Epoch 4/20
27/27 [==============================] - 1s 27ms/step - loss: 0.4208 - accuracy: 0.8026 - val_loss: 0.4073 - val_accuracy: 0.8184
Epoch 5/20
27/27 [==============================] - 1s 27ms/step - loss: 0.4172 - accuracy: 0.7944 - val_loss: 0.4369 - val_accuracy: 0.7859
Epoch 6/20
27/27 [==============================] - 1s 27ms/step - loss: 0.4114 - accuracy: 0.7933 - val_loss: 0.4437 - val_accuracy: 0.7886
Epoch 7/20
27/27 [==============================] - 1s 27ms/step - loss: 0.3956 - accuracy: 0.8177 - val_loss: 0.4369 - val_accuracy: 0.8049
Epoch 8/20
2

In [49]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.4573154151439667, 0.7669376730918884]

In [50]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.46
accuracy: 76.69
